Manipulando Arquivos de Dados
============
Neste exemplo você irá aprender a carregar arquivos XML.

**Antes de iniciar a atividade, caso opte por executar o script localmente, baixe os arquivos 'curriculo(*).xml' em sua máquina!**

Lembre-se de que para executar o código é necessário selecionar a célula que o contém e utilizar o botão "play" no menu superior. Você também pode mandar executar todas as células de uma única vez escolhendo a opção "Run all" no menu "Cell".

In [ ]:
import pandas as pd                    # pandas é uma biblioteca para manipulação de tabelas de dados
import xml.etree.ElementTree as ET     # ElementTree é uma biblioteca que carrega elementos de um arquivo XML

N_CVs = 1 # Número de arquivos de CVs a carregar (carrega 1 só para exemplificar)

# Carrega XML e mostra na tela dados principais do autor e também os artigos completos publicados
for i in range(1,N_CVs+1):
    arquivo = 'curriculo(' + str(i) + ').xml'
    print("Lendo arquivo: ", arquivo)
    doc_tree_root = (ET.parse(arquivo)).getroot()
    for elem in doc_tree_root:        
        if elem.tag == 'DADOS-GERAIS':
            print('Elemento: ' + elem.tag )
            print('Atributos: ') # + str(elem.attrib))
            for key, value in elem.items():
                print('\t'+ key + ': ' + value)
        if elem.tag == 'PRODUCAO-BIBLIOGRAFICA':
            print('Elemento: ' + elem.tag )
            trabalhos = elem.findall('./TRABALHOS-EM-EVENTOS/TRABALHO-EM-EVENTOS')
            for trabalho in trabalhos:
                dados_basicos = trabalho.find('DADOS-BASICOS-DO-TRABALHO').attrib
                detalhes = trabalho.find('DETALHAMENTO-DO-TRABALHO').attrib
                autores = trabalho.findall('AUTORES')
                if dados_basicos.get('NATUREZA') == 'COMPLETO':
                    print('\tArtigo: ' + dados_basicos.get('TITULO-DO-TRABALHO'))
                    for key, value in dados_basicos.items():
                        print('\t\t'+ key + ': ' + value)
                    for key, value in detalhes.items():
                        print('\t\t'+ key + ': ' + value)   
                    print('\t\tAutores: ')
                    for autor in autores:
                        for key, value in autor.items():
                            print('\t\t\t'+ key + ': ' + value)   

In [ ]:
import pandas as pd                    
import xml.etree.ElementTree as ET     

N_CVs = 1 
linhas = [] # uma linha para cada artigo lido

# Carrega CVs e cria um dataframe com os dados dos artigos lidos
for i in range(1,N_CVs+1):
    arquivo = 'curriculo(' + str(i) + ').xml'
    print("Lendo arquivo: ", arquivo)
    doc_tree_root = (ET.parse(arquivo)).getroot()
    for elem in doc_tree_root:                
        if elem.tag == 'PRODUCAO-BIBLIOGRAFICA':            
            trabalhos = elem.findall('./TRABALHOS-EM-EVENTOS/TRABALHO-EM-EVENTOS')
            for trabalho in trabalhos:
                dados_basicos = trabalho.find('DADOS-BASICOS-DO-TRABALHO').attrib
                detalhes = trabalho.find('DETALHAMENTO-DO-TRABALHO').attrib
                autores = trabalho.findall('AUTORES')
                if dados_basicos.get('NATUREZA') == 'COMPLETO':
                    print('\tArtigo encontrado: ' + dados_basicos.get('TITULO-DO-TRABALHO'))                    
                    dados = dict(dados_basicos)
                    dados.update(detalhes)
                    lst = []
                    lst_autores= {}
                    for autor in autores:                        
                        lst.append(autor.get('NOME-COMPLETO-DO-AUTOR'))                      
                    lst_autores['AUTORES'] = lst
                    dados.update(lst_autores)                    
                    linhas.append(dados)                                        
prodPPGC = pd.DataFrame(linhas) # df é o dataframe contendo todos os dados lidos

**Para saber quais são as colunas de informações lidas,  use o seguinte comando:**

In [ ]:
print(prodPPGC.columns)

**Para ver o conteúdo do dataframe, use o seguinte comando:**

In [ ]:
# prodPPGC   
prodPPGC.head() # mostra só os 5 primeiros, por questões de espaço (descomente a linha de cima, i.e., tire o head(), para mostrar todos)

**Vamos agora utilizar a bibliteca 'pandas' para: **
- **verificar quantos artigos (no total) são produzidos por ano, no programa PPGC;**
- **montar uma tabela que demonstre quantos artigos por tipo (periódico ou conferência) são publicados por ano.** 
- **mostrar um gráfico em barras que compare a produção de cada tipo, por ano);**
- **montar uma tabela que demonstre a quantidade total de artigos por QUALIS;**
- **extratificar a tabela anterior, demonstrando a quantidade de artigos em cada tipo de qualis, por ano, além de gráficos em linha que demonstrem a evolução da produção no período;**
- **montar uma tabela e um gráfico que demonstre informações estatísticas (mediana, moda, desvio padrão, etc.).** 


In [ ]:
# habilita a geração de gráficos no Jupyter
%matplotlib inline

In [ ]:
# Agrupa os registros por tipo de produção e coloca em 'prodPPGCByTipos':
prodPPGCByTipos = prodPPGC.groupby('CLASSIFICACAO-DO-EVENTO')

In [ ]:
# Mostra os 2 primeiros registros do dataset resultante 
# (como há 2 grupos de dados, um para cada tipo de produção, i.e., periódicos e conferências), 
# acaba mostrando 4, ou seja, 2 para cada grupo):
prodPPGCByTipos.head(2)

In [ ]:
# verifica o tamanho (quantidade de itens) de cada grupo e mostra gráfico em barras
prodPPGCByTipos.size().plot.bar(color=['red', 'blue'])

In [ ]:
# verifica o tamanho (quantidade de itens) de cada grupo e mostra gráfico em pizza
prodPPGCByTipos.size().plot.pie(colors=['red', 'blue'])

In [ ]:
# personalizando o gráfico
import matplotlib.pyplot as plot
plot.pie(prodPPGCByTipos.size(), colors=['red', 'blue'], labels=prodPPGCByTipos.groups.keys(), shadow=True,autopct='%1.1f%%')
plot.title('PUBLICACOES POR TIPO')

In [ ]:
# Agrupa por ano:
prodPPGCByAno = prodPPGC.groupby('ANO-DO-TRABALHO')

# conta quantos registros cada grupo (ano) possui
resultado = prodPPGCByAno.count()

# troca o nome da coluna 
resultado.rename(columns={'ANO-DE-REALIZACAO': 'Quantidade'}, inplace=True)

# mostra quantidade por ano, em forma de tabela
resultado[['Quantidade']]

In [ ]:
# Motra quantidade total de artigos por ano em um gráfico em barras
resultado['Quantidade'].plot.bar()

# também poderia ter feito direto: prodPPGCByAno.size().plot.bar()

In [ ]:
# mostra tabela com a evolução da quantidade por ano, extratificada por tipo 
resultado = prodPPGC.groupby(['ANO-DO-TRABALHO','CLASSIFICACAO-DO-EVENTO']).size().unstack()
resultado

In [ ]:
# Gráfico
resultado.plot(kind='bar', stacked=True)

** Tente descobrir o Qualis de cada publicação, usando os arquivos 'QualisConferencias.csv' (ou .xml) e 'QualisPeriodicos.csv' (ou .xml). Agregue essa informação à tabela. **

**Realize as demais atividades solicitadas, em resumo:**
- Crie uma **tabela (dataframe) de pesquisadores (professores)**, contendo ID (pode ser número inteiro incrementado automaticamente) e seus respectivos nomes completos, eliminar duplicatas, eliminar acentos, etc;
- Crie uma **tabela (dataframe) de linhas de pesquisa** (ver página do PPGC do Instituto de Informática) contendo ID (pode ser número inteiro incrementado automaticamente) e descrição da linha; 
- Crie uma **tabela (dataframe) de Eventos-Periódicos**, contendo ID (pode ser número inteiro incrementado automaticamente), nome do periódico ou evento, sigla (se tiver), ISSN (se tiver);
- Crie uma **tabela (dataframe) de QUALIS**, contendo ID (pode ser número inteiro incrementado automaticamente), nome do evento ou periódico, ano do QUALIS, sigla (se tiver) e ISSN (se tiver);
- Crie uma **tabela (dataframe) de fatos**, armazenando os dados das publicações e respectivos IDs das dimensões definidas;
- Crie uma **tabela (dataframe) de datas (ano)**, que pode também ter como atributo o quadriênio

**Salve todos os dataframes em arquivos separados**. Mais adiante, vamos usá-los para popular tabelas em um banco de dados (ou usar diretamente em uma ferramenta OLAP).

Para salvar os dataframes resultantes, utilize o comando to_csv('nome do arquivo.csv')!